# EDA: General

Este notebook tiene como objetivo limpiar, transformar, preprocesar y visualizar los datos de la unión de tablas realizadas en el ipynb anterior.

## 1. Libraries


In [ ]:
# Datos
import numpy as np
import pandas as pd

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from random import random

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from scipy import stats

# Evaluacion
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

#warnings
import warnings
warnings.filterwarnings("ignore")

#Extras
import os
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/CSV/creation/gb_union_weeks.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

### Optimización de los datos

In [ ]:
# Columnas a convertir a 'category', ya que son variables con pocos valores únicos (optimizar memoria)
col_cat= ['category', 'store', 'store_code', 'region', 'department']

# Aplicar el bucle para convertir las columnas a 'category'
for i in col_cat:
    df[i] = df[i].astype('category')

In [ ]:
df['date'] = pd.to_datetime(df["date"])

In [ ]:
df['quantity']=df['quantity'].astype('int16')

## 3. Preprocessing

In [ ]:
del df['year_week']

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
df.isnull().sum()

In [ ]:
# Ordenar el DataFrame por 'id' y 'date'
df = df.sort_values(by=['id', 'date'])

In [ ]:
df.head()

Este código recorre el DataFrame fila por fila. Si encuentra un 'quantity' distinto de cero para un 'id', pasa al siguiente 'id'. Si encuentra un 'quantity' igual a cero y aún no ha encontrado un valor distinto de cero para ese 'id', entonces reemplaza el valor de 'sell_price' por -99.

Este código se ha realizado porque se observaron que algunos productos al inicio de los datos no tenían ventas. Por lo tanto, se concluyó que salieron a mercado un tiempo más tarde, pero que sufieron épocas que no fueron adquiridos por los clientes finales.


In [ ]:
previous_id = None
for index, row in df.iterrows():
    current_id = row['id']
    if current_id != previous_id:
        previous_id = current_id
        found_nonzero = False
    if row['quantity'] != 0:
        found_nonzero = True
    if row['quantity'] == 0 and not found_nonzero:
        df.at[index, 'sell_price'] = -99

print(df)

In [ ]:
df.isnull().sum()

In [ ]:
df["sell_price"] = df.groupby("id", group_keys = False)["sell_price"].apply(
    lambda series: series.backfill().ffill()
)

In [ ]:
df['sell_price'] = df['sell_price'].round(2)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df[df['sell_price'] != -99]

print(df)

In [ ]:
# Reasignar los índices después de eliminar las filas
df.reset_index(drop=True, inplace=True)

## 4. Feature engineering

In [ ]:
for col in ['date']:
  df['week'] = df[col].dt.week
  df['month'] = df[col].dt.month
  df['year'] = df[col].dt.year

df.head()

In [ ]:
# Columnas a convertir a 'category', ya que son variables con pocos valores únicos (optimizar memoria)
col_uint= ['week', 'month',]

# Aplicar el bucle para convertir las columnas a 'category'
for i in col_uint:
    df[i] = df[i].astype('uint8')

In [ ]:
df['year']=df['year'].astype('int16')

In [ ]:
df['revenue'] = df['quantity'] * df['sell_price']
df.head()

In [ ]:
df.info()

In [ ]:
df['revenue']=df['revenue'].astype('float32')

In [ ]:
df.info()

## 5. EDA: General

El objetivo es visualizar cómo se comportan las diferentes features para posteriormente saber qué gráficos realizar en Power BI para sacar insights significativos.

In [ ]:
# Agrupa los datos por fecha y suma las ventas
precio_totales = df.groupby('date')['sell_price'].sum()

plt.figure(figsize=(10, 6))
plt.plot(precio_totales.index, precio_totales.values, linestyle='-')
plt.title('Precio a lo largo del tiempo')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Agrupa los datos por fecha y suma las ventas
ventas_totales = df.groupby('date')['revenue'].sum()

plt.figure(figsize=(10, 6))
plt.plot(ventas_totales.index, ventas_totales.values, linestyle='-')
plt.title('Ventas totales a lo largo del tiempo')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
ventas_por_mes = df.groupby('month')['revenue'].sum()

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
plt.plot(ventas_por_mes.index, ventas_por_mes.values, marker='o')
plt.title('Evolución de las ventas mensuales')
plt.xticks(ventas_por_mes.index)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Crear el diagrama de dispersión
plt.figure(figsize=(10, 6))
plt.scatter(df['sell_price'], df['quantity'], alpha=0.5)
plt.title('Relación entre precio de venta y cantidad vendida')
plt.xlabel('Precio')
plt.ylabel('Cantidad')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
ventas_por_region = df.pivot_table(index='region', columns='year', values='revenue', aggfunc='sum')
sns.heatmap(ventas_por_region, cmap='Blues')
plt.title('Ventas por región anuales')
plt.show()

In [ ]:
ventas_por_categoria = df.groupby('category')['revenue'].sum()

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
ventas_por_categoria.plot(kind='bar')
plt.title('Cantidad de ventas por categoría')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
ventas_por_departamento_anio = df.pivot_table(index='year', columns='department', values='revenue', aggfunc='sum')
ventas_por_departamento_anio.plot(kind='line', marker='o')
plt.xlabel('Año')
plt.ylabel('Cantidad de ventas')
plt.title('Comparación de las ventas anuales por departamentos')
plt.legend(title='Departamento', loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
#examinar la popularidad de los productos e identificar los más vendidos en cada ciudad.
ventas_ciudad_producto = df.groupby(['region', 'item'])['quantity'].sum().reset_index()

In [ ]:
#productos más vendidos en cada ciudad
prod_mas_vendidos = ventas_ciudad_producto.groupby('region').apply(lambda x: x.nlargest(5, 'quantity')).reset_index(drop=True)

In [ ]:
#grafico
plt.figure(figsize=(12, 8))
sns.barplot(x='quantity', y='item', hue='region',  data=prod_mas_vendidos)
plt.title('Top 5 de productos más vendidos en cada ciudad')
plt.xlabel('Unidades vendidas')
plt.ylabel('Producto')
plt.legend(title='Ciudad', loc='upper right')
plt.show()